___
# **Pipeline de Sistema de Recomendación (MVP)**

Este proyecto implementa un sistema de recomendación basado en contenido. 

Utiliza Procesamiento de Lenguaje Natural (NLP) para analizar las descripciones de los productos y sugerir artículos similares.

- `products.csv`: Es la base de datos de productos (Materia prima).
- `pipeline_entrenamiento.py`: Es el código que aprende y crea el modelo
- `modelo_recomendacion_v1.pkl`: Se genera automáticamente, es el cerebro del modelo.

___

In [16]:
# Librerías
import pandas as pd
import pickle # para guardar archivos binarios (el cerebro guardado)
from sklearn.feature_extraction.text import TfidfVectorizer # para convertir texto en vectores númericos
from sklearn.metrics.pairwise import linear_kernel # para medir la similitud entre productos


___
Preparación de Datos (ETL)

In [17]:
# Definimos una función para limpiar y preparar
def cargar_y_preparar_datos(ruta_archivo):
    try: # Cargar CSV
        df = pd.read_csv(ruta_archivo)
    except FileNotFoundError:
        return None, "Archivo no encontrado"

    # Rellenar vacíos con texto vacío (para que no falle la suma de textos)
    columnas_clave = ['ProductName', 'Brand', 'Category', 'SubCategory']
    for col in columnas_clave:
        df[col] = df[col].fillna('')

    # Creamos un perfil de texto
    # Unimos todo en una sola columna
    def unir_columnas(fila):
        return f"{fila['Brand']} {fila['Category']} {fila['SubCategory']} {fila['ProductName']}"

    df['perfil_texto'] = df.apply(unir_columnas, axis=1)
    
    # Limpieza minúsculas y quitar espacios
    df['perfil_texto'] = df['perfil_texto'].str.lower().str.strip()
    print("Datos listos. 'perfil_texto'.")
    print("---"*10)
    return df

# Prueba de la función...
df_productos = cargar_y_preparar_datos("products.csv")

print(f"\nPerfil creado: \n{df_productos['perfil_texto'].iloc[0:5]}") 

Datos listos. 'perfil_texto'.
------------------------------

Perfil creado: 
0    fresho fruits & vegetables potato, onion & tom...
1    fresho fruits & vegetables potato, onion & tom...
2    fresho eggs, meat & fish farm eggs farm eggs -...
3    fresho fruits & vegetables potato, onion & tom...
4    fresho fruits & vegetables potato, onion & tom...
Name: perfil_texto, dtype: object


___
Entrenamiento del Pipeline

In [18]:
# Función para entrenar el pipeline
def entrenar_pipeline(df):
    # Vectorización de texto
    # stop_words='english' elimina palabras que no aportan valor (el, la, the, and)
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Transformamos el texto a matriz de números
    matriz_tfidf = tfidf.fit_transform(df['perfil_texto'])
    print(f"Matriz creada: {matriz_tfidf.shape} (Productos x Palabras)")
    print("---"*10)

    # Cálculo de similitud. Calculamos la distancia de todos contra todos
    matriz_similitud = linear_kernel(matriz_tfidf, matriz_tfidf)
    print("Matriz de similitud calculada.")
    print("---"*10)

    # Guardamos la matriz en un archivo .pkl (pickle)
    nombre_archivo = 'modelo_recomendacion_v1.pkl'
    with open(nombre_archivo, 'wb') as f:
        pickle.dump(matriz_similitud, f)
    print(f"Modelo guardado en '{nombre_archivo}'")
    return matriz_similitud

# Entrenamos el pipeline
matriz_entrenada = entrenar_pipeline(df_productos)

Matriz creada: (2000, 2636) (Productos x Palabras)
------------------------------
Matriz de similitud calculada.
------------------------------
Modelo guardado en 'modelo_recomendacion_v1.pkl'


___
Uso del Modelo (Inferencia)

In [19]:
# Función para obtener recomendaciones
def obtener_recomendaciones(nombre_producto, df, matriz_similitud):
    print(f"\nRecomendaciones para: '{nombre_producto}'...")
    
    # Crear índice para buscar rápido
    indices = pd.Series(df.index, index=df['ProductName']).drop_duplicates()

    # Validar que el producto exista
    if nombre_producto not in indices:
        return "El producto no existe, intente con otro producto."

    # Obtener el ID numérico del producto
    idx = indices[nombre_producto]

    # Obtener los puntajes de similitud de ese producto
    puntajes = list(enumerate(matriz_similitud[idx]))

    # Ordenar de mayor a menor similitud
    puntajes = sorted(puntajes, key=lambda x: x[1], reverse=True)

    # Tomar los 3 mejores (saltamos el [0] porque es el mismo producto)
    top_3 = puntajes[1:4]
    
    # Recuperar la información del DataFrame original
    indices_recomendados = [i[0] for i in top_3]
    resultado = df[['ProductName', 'Brand', 'Category']].iloc[indices_recomendados]
    
    return resultado

___
Prueba de pipeline

In [20]:
# Prueba
# Elegimos un producto al azar para probar
producto_ejemplo = df_productos['ProductName'].iloc[0] # <-- cambia el producto  


# Llamamos a la función
recomendaciones = obtener_recomendaciones(producto_ejemplo, df_productos, matriz_entrenada)
print("---"*10)
print("\nRecomendaciones:")
display(recomendaciones) 


Recomendaciones para: 'Onion (Loose)'...
------------------------------

Recomendaciones:


,ProductName,Brand,Category
1,Onion,Fresho,Fruits & Vegetables
3,Potato (Loose),Fresho,Fruits & Vegetables
4,Potato,Fresho,Fruits & Vegetables
